In [2]:
# Mount google drive and setup paths
import sys
 
# mount google drive 
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
 
# specify path name to import py file
folder_name = 'Jupyter Notebooks/Quantitative Finance'
sys.path.append("drive/My Drive/" + folder_name)

Mounted at /content/drive/


In [0]:
# import necessary libraries
import numpy as np
from Option import *
from Binomial_tree import *

[Here justifies the two formulas for up and down factors will converge to the same option price](https://quant.stackexchange.com/questions/32524/difference-in-formulas-for-u-d-in-binomial-trees)

A recombinant tree is a tree where an up-move followed by a down-move is the same as down-move followed by an up-move. 
It follows that
$$S_n = S_0 \times u^{N_u} d^{N_d}$$
where $N_u$ and $N_d$ are the number of up-move and down-move respectively.

Recombining trees are a standard method for pricing options. In this project, we try them out for some simple options.
Since a recombining tree makes use of the fact that an up-move followed by a down-move is the same as down-move followed by an up-move, this keeps the total number of node tractable; otherwise the number of nodes grows exponentially. A tree should be implemented purely in the risk-neutral world, the real-world tree is useful for justifying risk-neutral pricing but not for actually doing the pricing. We also work with the log as the geometry is simple.

We wish to price an option under geometric Brownian motion. 
So, we discretize
$$d(\log S) = \left( r -d - \frac{1}{2}\sigma^2 \right)dt + \sigma dW.$$
We divide time into $N$ steps of length $\Delta t.$
For a binomial tree, at each step, $\log S$, goes either up or down by $\sigma \sqrt{\Delta t}$ and increases by $(r - 0.5 \sigma^2)\Delta t.$
One can therefore easily construct the set of all possible nodes across $N$ steps, and work out how they relate to each other.
In other words, the equation is 
$$\log S_{j\Delta t} = \log S_{(j-1)\Delta t} + \left( r - d - \frac{1}{2} \sigma^2 \right)\Delta t + \sigma \sqrt{\Delta t} Z_j \quad \text{for all } j=1,2,...,N$$
where $Z_j$ is a Bernoulli random variable on $\{-1,1\}$ with $\mathbb{P}(Z_j = -1) = \mathbb{P}(Z_j = 1) = \frac{1}{2}.$

To read more on binomal tree option pricing, please refer to [wikipedia](https://en.wikipedia.org/wiki/Binomial_options_pricing_model).

## Cox, Ross  & Rubinstein (CRR) tree
Cox, Ross  & Rubinstein (CRR) tree formulas to calculate the up and down multipliers are 
$$u = e^{\sigma\sqrt{\Delta t}} \quad \text{and} \quad d = e^{-\sigma\sqrt{\Delta t}} =\frac{1}{u}.$$
For motivation, please refer to this post at [QFSE](https://quant.stackexchange.com/questions/45293/whats-the-logic-behind-binomial-model-ups-and-downs).
[Here](https://drive.google.com/file/d/13bBpLtKSsLzOZ-O5Cxdb4wc_fkTMYB-t/view) is a copy of the research paper of CRR.

Note that CRR tree is recombinant.

In [5]:
S0 = 100
K = 100
sigma = 0.2
r = 0.1
d = 0.05
T = 2

# Compare tree price by crr and gbm with Black-Scholes analytical pricing
num_sim = 11
tree = BinomialTree(S0, r, d, sigma, T)
payoff = lambda x: np.maximum(x - K, 0)

print(tree.option_price(num_sim, payoff = payoff, model = 'crr'))
print(tree.option_price(num_sim, payoff = payoff, model = 'gbm'))
Option(S0, K, r, d, sigma, T).european_call()

[13.429046613293686, 13.978589589621329, 14.27838570297846, 14.43373069112047, 14.512575746767117, 14.552262545965716, 14.5721681546458, 14.582136016946038, 14.587123650176425, 14.589618384483256, 14.590865980082016]
[14.502344444882333, 14.726551251167768, 14.764368165625099, 14.72212196384914, 14.65683628252714, 14.593095607216847, 14.596392437401656, 14.598091935829931, 14.596059082840577, 14.594067056691532, 14.592402997834496]


14.592113727584305

# Pricing rules
A tree is well-suited to pricing an option whose value can be written as a function of the current spot and the option's expected value in the function.
In practical terms, we can price options whose value can be specified at a node as a function of spot at the current node, and the discounted average of the values at its two daughter nodes.
Note the discounted average will be 
$$e^{-r\Delta t} \left(\tilde{p} \text{Value-up} + (1-\tilde{p})\text{Value-down} \right)$$
where $\tilde{p}$ is the risk-neutral probability.
Work out what the rule for computing the price at a node is for each of the following derivatives.

(i) a vanilla call option,

(ii) a forward,

(iii) a put option,

(iv) a down-and-out call or put option,

(v) an American option,

(vi) a digital option.


# Pricing on the tree
Now implement the binomial tree and apply it to the pricing of each of the above. If you are using an object-oriented programming langugage, keep the definitions of the tree and of the option rule as separated as possible so you can plug in various different rules without recoding the tree. The option will be specified by the rule for its value at a node, and by its final payoff.

Plot the value of each option as a function of the number of steps for reasonable parameters (e.g. spot 100, strike 100, $r = 0.05$, $\sigma = 0.1$ and $T =2$.) Check the answers you get against analytic formula or another pricing method.

For which options does the price oscillate? 
Compare the rate of convergence with that obtained by taking the sequence obtained by averaging the $N$-step oruce with the $(N+1)$-step price.

Price an option with a strike that puts the option far out-of-the-money. 
How does the speed of convergence change?


# Trinomial trees
Repeat everything with a trinomial tree.
Compare the rate of convergence both as a function of the number of steps and as a function of the number of nodes. 
See if you can get improved convergence rates by adapting the position of the nodes to specific properties of the derivative product.